In [29]:
## Class Definitions

class Quote:
    def __init__(self, quote, category):
        self.quote = quote
        self.category = category
        

In [30]:
import numpy as np
import pandas as pd

df = pd.read_csv('./data/reduced_quotes.csv')
reduced_df = df.sample(frac=0.1, random_state=42)  # Keep 50% of the data
reduced_df.to_csv('./data/reduced_reduced_quotes.csv', index=False)
reduced_df = reduced_df.dropna()
quotes = []

reduced_df['processed_category'] = reduced_df['category'].apply(lambda x: ' '.join(str(x).split(', ')))

print(reduced_df.head(5))

for _, row in reduced_df.iterrows():
    quotes.append(Quote(row['quote'], row['processed_category']))

print(len(quotes))


                                                   quote  \
3835   Despite all my rageI am still just a rat in th...   
3578   The most important experiences a man can have ...   
17679  I can feel myself gently slipping away into a ...   
31388  It is tempting to write the history of technol...   
12801  He said he'd never remarry, because he'd never...   

                                   author                        category  \
3835                         Billy Corgan  anger, cages, life, rage, rats   
3578         Paulo Coelho, Eleven Minutes                         courage   
17679  Rak Razam, Aya: A Shamanic Odyssey    spirituality, transformation   
31388                      the microscope                    the airplane   
12801        Alexandra Bracken, Passenger                       true-love   

                processed_category  
3835    anger cages life rage rats  
3578                       courage  
17679  spirituality transformation  
31388                 the ai

In [31]:

from sklearn.model_selection import train_test_split

print(len(quotes))

training, test = train_test_split(quotes, test_size=0.33, random_state=42, shuffle=True)

print(len(training))

x_train = [q.quote for q in training]
y_train = [q.category for q in training]

x_train = ["" if pd.isna(text) else str(text) for text in x_train]


x_test = [q.quote for q in test]
y_test = [q.category for q in test]

print(x_test[0])
print(y_test[0])


3240
2170
I own my past, it hasn't been grande' But it's had some pretty great moments. I own my movements of now, it isn't what I've dreamt, but I'm closer than I was before.I own my future, it is going to test me, But I trust I have the strength to pull through. Life isn't what happens to us, but what we choose to become.
experience gratitude growth inspiring journey lessons life pathway seek transformational truth


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
train_x_vectors = tfidf_vectorizer.fit_transform(x_train)

In [33]:
print(train_x_vectors.shape)
print(train_x_vectors[0].toarray())
print(tfidf_vectorizer.get_feature_names_out())


(2170, 9575)
[[0. 0. 0. ... 0. 0. 0.]]
['000' '10' '100' ... 'zone' 'zones' 'zoophilism']


In [38]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(train_x_vectors, dense_output=False)

print(similarity_matrix.shape)


(2170, 2170)


In [55]:
## Recommending Similar Quotes

def recommend_similar_quotes(quote_index, num_recommendations=4):
    print("Quote Index:", quote_index)
    print("Number of Recommendations:", num_recommendations)

    similarity_row = similarity_matrix[quote_index].toarray().flatten()
    similar_indices = np.argsort(similarity_row)[-num_recommendations - 1:][::-1]
    print("Similar Indices:", similar_indices)

    similar_quotes = [quotes[i].quote for i in similar_indices if i != quote_index]
    return similar_quotes

print(recommend_similar_quotes(2))

Quote Index: 2
Number of Recommendations: 4
Similar Indices: [   2  423 1121 1458 1514]
["Middle age is when you've met so many people that every new person you meet reminds you of someone else.", 'Work for legal reform takes place only within the framework of the social form created by the last revolution.', "To embrace the message of Christmas is to throw off my hedonistic rebellion and bow before the chafing reality that I can't save myself, and in that very act to be suddenly taken aback in that I've stumbled upon the very freedom I've longed for in the very place I'd least expected it.", "One of my biggest pet peeves is that I just don't like it when characters do things that are funny to the writer, but you don't know why they're doing it and it doesn't make any sense."]


In [ ]:
## 